# Movies Imformation - Retrieval  

## Librarys

In [33]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import scipy.sparse

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from nltk.stem import WordNetLemmatizer
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from num2words import num2words
from sklearn.metrics.pairwise import linear_kernel
from ast import literal_eval

title = "stories"
alpha = 0.4

## Import DataSet

In [98]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
cast_crew_text = pd.read_csv('../the-movies-dataset/cast_crew_text.csv')
smd['description'] = smd['description'].fillna('')
smd['title'] = smd['title'].fillna('')
N = len(smd)

In [99]:
cast_crew_text

,soup
0,Woody (voice) Buzz Lightyear (voice) Mr. Potat...
1,Alan Parrish Samuel Alan Parrish / Van Pelt Ju...
2,Max Goldman John Gustafson Ariel Gustafson Mar...
3,Savannah 'Vannah' Jackson Bernadine 'Bernie' H...
4,George Banks Nina Banks Franck Eggelhoffer Ann...
...,...
9020,Fin Shepard April Wexler Colonel Gilbert Shepa...
9021,Henry Cobb Karen Potter Mike Potter Dorothy Co...
9022,Rustom Pavri Cynthia Rustom Pavri Priti Makhij...
9023,"Sarman Chaani Maham, Senate Chief Moonja, Maha..."


In [103]:
smd['soup'] = cast_crew_text['soup']

In [101]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9025 entries, 0 to 9024
Data columns (total 27 columns):
adult                    9025 non-null bool
belongs_to_collection    1670 non-null object
budget                   9025 non-null int64
genres                   9025 non-null object
homepage                 1954 non-null object
id                       9025 non-null int64
imdb_id                  9025 non-null object
original_language        9025 non-null object
original_title           9025 non-null object
overview                 9013 non-null object
popularity               9025 non-null float64
poster_path              9022 non-null object
production_companies     9025 non-null object
production_countries     9025 non-null object
release_date             9025 non-null object
revenue                  9025 non-null float64
runtime                  9025 non-null float64
spoken_languages         9025 non-null object
status                   9023 non-null object
tagline             

## Preprocessing

### Lowercase

In [82]:
def convert_lower_case(data):
    return np.char.lower(data)

### Stopwords

In [83]:
# Stop words are the most commonly occurring words which don’t give any
# additional value to the document vector. in-fact removing these will
# increase computation and space efficiency. 

In [84]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [85]:
def remove_stop_words(data):
    new_text = ""
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))

    for word in words:
        if word not in stop_words and len(word) > 1:
            new_text = new_text + " " + word
        
    return new_text        

### Punctuation

In [86]:
# (Punctuation - dấu chấm câu)are the unnecessary symbols that are in
# our corpus documents, we should be little careful with what we are 
# doing with this. There might be few problems such as U.S — us 
# “United Stated” being converted to “us” after the preprocessing. 
# hyphen and should usually be dealt little carefully. but for this 
# problem statement we are just going to remove these

In [87]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [88]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

### Steamming

In [89]:
def stemming(data):
    new_text = ""
    words = word_tokenize(str(data))
    stemmer = PorterStemmer()
    
    for word in words:
        new_text = new_text + " " + stemmer.stem(word)
    return new_text

In [90]:
data = "The striped bats are hanging on their feet for best"
stemming(data)

' the stripe bat are hang on their feet for best'

### Lemmatizer words

#### Get word pos 

In [91]:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

#### Lemmatizer words

In [92]:
def lemmatizer_words(data):
    new_text = ""
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(str(data))
   
    lemmatizer_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]

    for word in lemmatizer_text:
        new_text = new_text + " " + word
    return  new_text

### Convert Number 

In [93]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [94]:
data = "123"
convert_numbers(data)

array(' one hundred and twenty three', dtype='<U29')

### Combine all Preprocess step

In [95]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data)
    
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [109]:
processed_text = []
processed_title = []

for index, row in smd.iterrows():
    description = row['description'] + ' ' + str(row['soup'])
    title = row['title']   
    
    processed_text.append(word_tokenize(str(preprocess(description))))
    processed_title.append(word_tokenize(str(preprocess(title))))


KeyboardInterrupt: 

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

## Create Corpus Bag 

In [ ]:
# Mục đích là để tạo ra một bộ từ điển liệt kê tất cả các các từ và danh sách các bộ phim có sự xuất hiện của nó 


DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i} #Trừ khi trong DF[w] có tồn tại i
    
    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}


In [ ]:
# tính tần xuất xuất hiện của từ đó trên toàn bộ danh sách bộ phim 
for i in DF:
    DF[i] = len(DF[i])

In [ ]:
total_vocab_size = len(DF)
total_vocab_size

In [ ]:
total_vocab = [x for x in DF]
total_vocab[:10]

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### TF Cosine Similarity Ranking

In [105]:
# hàm tính đố tương đồng giữa 2 vector 
def cosine_sim_calculate(a,b):
    cos_sim = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

#### Vectorizing tf-idf

In [106]:
# lập danh sách các vector 
# ma trận D ban đầu có giá trị toàn bằng 0, sau đó chúng ta sẽ duyệt tf_idf và trám các giá trị có tồn tại vào D
D = np.load("tf_idf_vector_matrix.npy")

In [107]:
D.shape

(9025, 22336)

In [108]:
total_vocab[:10]

NameError: name 'total_vocab' is not defined

In [ ]:
# tạo vector từ query, nếu từ nào không xuất hiện thì tần suất xuất hiện của nó bằng 0
def gen_vector(tokens):
    
    Q = np.zeros(total_vocab_size)
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    
    for token in np.unique(tokens):
        try:
            ind = total_vocab.index(token)
            tf = counter[token]/words_count
            df = doc_freq(token)
            idf = math.log(N/(df + 1))
            Q[ind] = tf*idf
        except:
            pass
    
    return Q

#### Cosine similarity with query string 

In [ ]:
#query = "Dark night"

#preprocessed_query = preprocess(query)
#tokens = word_tokenize(str(preprocessed_query))
#array =  gen_vector(tokens)

#for i in array: 
#    print(i)

In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\n Query:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim_calculate(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    for i in out:
        print(smd.iloc[i]['title'])
    
    

In [ ]:
cosine_similarity(10, "The Dark Knight")

In [35]:
cosine_similarity(10, "joker gotham city")

Cosine Similarity

 Query: joker gotham city

['joker', 'gotham', 'citi']

Batman
The Dark Knight
In the City of Sylvia
The Dark Knight Rises
Batman: Year One
Joker
Batman: Mask of the Phantasm
Batman Returns
Sin City
Sound City


In [36]:
Q = cosine_similarity(10, "The Godfather")

Cosine Similarity

 Query: The Godfather

['godfath']

The Godfather: Part III
Tokyo Godfathers
The Godfather
Bright Eyes
The Godfather: Part II
Things to Do in Denver When You're Dead
Jane Austen's Mafia!
Free Enterprise
The Freshman
Cinderfella


In [44]:
D1 = scipy.sparse.load_npz('test_sparse.npz')

In [66]:
D[2]

array([0., 0., 0., ..., 0., 0., 0.])

In [68]:
D1[2,:].toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [73]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\n Query:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D1:
        d_cosines.append(cosine_sim_calculate(query_vector, d.toarray()[0]))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    for i in out:
        print(smd.iloc[i]['title'])
    
    

In [74]:
cosine_similarity(10, "The Dark Knight")

Cosine Similarity

 Query: The Dark Knight

['dark', 'knight']

Last Knights
The Dark Knight Rises
The Hollywood Knights
A Knight's Tale
The Seeker: The Dark Is Rising
Dark Water
The Dark Knight
Dark Water
Black Knight
A Shot in the Dark
